In [6]:
#import librariesImport
# import gedlibpy
import networkx as nx
import numpy as np
from torch_geometric.datasets import TUDataset
from torch_geometric.utils import to_networkx
from torch_geometric.utils import from_networkx
import functools
from tqdm import tqdm
import random
import time
import os
import torch
from multiprocessing import Pool
from matplotlib import pyplot as plt
from torch_geometric.datasets import Planetoid
import signal


In [7]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

In [8]:
dest_folder = "./data_wo_node_attr/"
if not os.path.exists(dest_folder):
    os.makedirs(dest_folder)
    
# Filter graphs with more than max_num_nodes nodes
max_num_nodes = 20
NUM_WORKER = 16

In [9]:
# All Methods for filtering graphs

class TimeoutError(Exception):
    pass

def handler(signum, frame):
    raise TimeoutError()

def set_x(data_obj):
    data_obj.x = torch.ones(data_obj.num_nodes, 1)
    return data_obj

def node_match(n1, n2):
    return np.argmax(n1['x']) == np.argmax(n2['x'])

def node_match_multi_hot(n1, n2):
    return np.array_equal(n1['x'], n2['x'])

def get_unique_graphs_using_nx(nx_graphs, timeout_duration = 15, node_match_func = None, max_unique_cnt = None):
    # Filter Isomorphic graphs using nx.is_isomorphic
    unique_graphs = [nx_graphs[0]]
    if max_unique_cnt == None:
        max_unique_cnt = len(nx_graphs)

    if node_match_func == None:
        node_match_func = node_match
    
    signal.signal(signal.SIGALRM, handler) 

    print("Filtering isomorphic graphs...")
    for i in tqdm(range(1, len(nx_graphs))):
        unique = True
        if(len(unique_graphs) >= max_unique_cnt):
            break
        for j in range(len(unique_graphs)):
            g1 = nx_graphs[i]
            g2 = unique_graphs[j]
            if g1.number_of_nodes() > g2.number_of_nodes():
                g1, g2 = g2, g1

            signal.alarm(timeout_duration)
            try:
                if nx.is_isomorphic(g1, g2, node_match=node_match):
                    #print("Graph ", i, " is isomorphic to graph ", j)
                    unique = False
                    break
            except TimeoutError:
                print(f"Timeout for graph {i}")
                unique = False
                break
            finally:
                signal.alarm(0)
        if unique:
            #print("Including graph ", i, " in unique_graphs")
            unique_graphs.append(nx_graphs[i])
    print(f"Total number of Unique graphs: {len(unique_graphs)}")
    return unique_graphs


def split_and_dump_graphs(nx_graphs, data_path = None, DATASET = ""):
    print("Splitting and dumping graphs...")
    if data_path == None:
        print("ERROR: Data path not provided")
        return
    
    random.seed(0)
    random.shuffle(nx_graphs)    # Shuffle the list of graphs
    train_size = int(0.6*len(nx_graphs))
    val_size = int(0.2*len(nx_graphs))
    test_size = len(nx_graphs) - train_size - val_size

    train_graphs = nx_graphs[:train_size]
    val_graphs = nx_graphs[train_size:train_size+val_size]
    test_graphs = nx_graphs[train_size+val_size:]
    print(f"Train: {len(train_graphs)}, Val: {len(val_graphs)}, Test: {len(test_graphs)}")


    # Dump Non-isomorphic Graphs 
    train_data = []
    for graph in train_graphs:
        data = from_networkx(graph, group_node_attrs = ['x'])
        train_data.append(data)
    
    print("sample:", train_data[0])
    torch.save(train_data, data_path + "train.pt")
    print(f"saved {DATASET} into {data_path}train.pt")

    val_data = []
    for graph in val_graphs:
        data = from_networkx(graph, group_node_attrs = ['x'])
        val_data.append(data)
    #print(val_data[0].x)
    torch.save(val_data, data_path + "val.pt")
    print(f"saved {DATASET} into {data_path}val.pt")

    test_data = []
    for graph in test_graphs:
        data = from_networkx(graph, group_node_attrs = ['x'])
        test_data.append(data)
    torch.save(test_data, data_path + "test.pt")
    print(f"saved {DATASET} into {data_path}test.pt")


def check_leakage(data_path, node_match_fun = None, full_check = False):
    print("Checking for leakage...")
    print("loading saved splits from ", data_path)
    train_data = torch.load(data_path + "/train.pt")
    val_data = torch.load(data_path + "/val.pt")
    test_data = torch.load(data_path + "/test.pt")

    train_nx = list(map(functools.partial(to_networkx, to_undirected=True, node_attrs = ['x']), train_data))
    val_nx = list(map(functools.partial(to_networkx, to_undirected=True, node_attrs = ['x']), val_data))
    test_nx = list(map(functools.partial(to_networkx, to_undirected=True, node_attrs = ['x']), test_data))
    
    print("Number of graphs in train: ", len(train_nx), " Val: ", len(val_nx), " Test: ", len(test_nx))

    if node_match_fun == None:
        node_match_fun = node_match

    if full_check:
        cnt = 0
        for i in tqdm(range(len(train_nx))):
            for j in range(len(train_nx)):
                if i == j:
                    continue
                if nx.is_isomorphic(train_nx[i], train_nx[j], node_match=node_match_fun):
                    #print("Isomorphic graph found between train and val")
                    cnt += 1
                    break
        print("Number of duplicate inside train: ", cnt)
        
        cnt = 0
        for i in tqdm(range(len(val_nx))):
            for j in range(len(val_nx)):
                if i == j:
                    continue
                if nx.is_isomorphic(val_nx[i], val_nx[j], node_match=node_match_fun):
                    #print("Isomorphic graph found between train and val")
                    cnt += 1
                    break
        print("Number of duplicates inside val:", cnt)
        
        cnt = 0
        for i in tqdm(range(len(test_nx))):
            for j in range(len(test_nx)):
                if i == j:
                    continue
                if nx.is_isomorphic(test_nx[i], test_nx[j], node_match=node_match_fun):
                    #print("Isomorphic graph found between train and val")
                    cnt += 1
                    break
        print("Number of duplicates inside test:", cnt)


    cnt = 0
    for val_g in tqdm(val_nx):
        for train_g in train_nx:
            if nx.is_isomorphic(train_g, val_g, node_match=node_match_fun):
                #print("Isomorphic graph found between train and val")
                cnt += 1
                break
    print("Number of leaks in Val from train: ",cnt)

    cnt = 0
    for test_g in tqdm(test_nx):
        for train_g in train_nx:
            if nx.is_isomorphic(train_g, test_g, node_match=node_match_fun):
                #print("Isomorphic graph found between train and val")
                cnt += 1
                break
    print("Number of leaks in test from train: ",cnt)

def generate_uniqe_graphs(graphs, DATASET, node_labeled = False, node_is_multihot = False, max_unique_cnt = None, check = False):
    """
        graphs: list of torch_geometric.data.Data objects
            data[i].x: Node features of graph i
        DATASET: Name of the dataset

    """

    if not node_labeled and node_is_multihot:
        print("ERROR: Node cannot be multihot and not labeled at the same time")
        return
    
    data_path = dest_folder + DATASET + '/'
    if not os.path.isdir(data_path):
        os.mkdir(data_path)

    print("Total number of graphs: ", len(graphs))
    print("Selecting Small Graphs...")
    graphs = list(filter(lambda x: x.num_nodes <= max_num_nodes, graphs))
    print(f"Graphs with at most {max_num_nodes} nodes: {len(graphs)}")

    if not node_labeled:
        graphs = list(map(set_x, graphs))
    
    nx_graphs = list(map(functools.partial(to_networkx, to_undirected=True, node_attrs = ['x']), graphs))

    if node_is_multihot:
        node_match_func = node_match_multi_hot
    else:
        node_match_func = node_match

    unique_nx_graphs = get_unique_graphs_using_nx(nx_graphs, node_match_func = node_match_func, max_unique_cnt = max_unique_cnt)
    
    sizes = list(map(lambda x: x.number_of_nodes(), unique_nx_graphs))
    print("Maximum number of nodes in the dataset: ", max(sizes))
    split_and_dump_graphs(unique_nx_graphs, data_path, DATASET)
    if check:
        check_leakage(data_path, full_check = False)

# AIDS

In [10]:
DATASET = 'AIDS'
data = TUDataset(root="/tmp/", name=DATASET)
node_labeled = False

In [11]:
generate_uniqe_graphs(data, DATASET, node_labeled)

Total number of graphs:  2000
Selecting Small Graphs...
Graphs with at most 20 nodes: 1666
Filtering isomorphic graphs...


 13%|█▎        | 224/1665 [00:00<00:02, 554.79it/s] 

100%|██████████| 1665/1665 [00:09<00:00, 174.52it/s]


Total number of Unique graphs: 911
Maximum number of nodes in the dataset:  20
Splitting and dumping graphs...
Train: 546, Val: 182, Test: 183
sample: Data(edge_index=[2, 20], x=[11, 1])
saved AIDS into ./data_wo_node_attr/AIDS/train.pt
saved AIDS into ./data_wo_node_attr/AIDS/val.pt
saved AIDS into ./data_wo_node_attr/AIDS/test.pt


In [12]:
check_leakage(dest_folder + '/AIDS/', full_check = False)

Checking for leakage...
loading saved splits from  ./data_wo_node_attr//AIDS/
Number of graphs in train:  546  Val:  182  Test:  183


100%|██████████| 182/182 [00:01<00:00, 134.84it/s]


Number of leaks in Val from train:  0


100%|██████████| 183/183 [00:01<00:00, 132.68it/s]

Number of leaks in test from train:  0


In [13]:
dummy = torch.load(dest_folder + '/AIDS/train.pt')
print(dummy[0])

Data(edge_index=[2, 20], x=[11, 1])


# LINUX

In [28]:
from  torch_geometric.datasets.ged_dataset import GEDDataset

In [29]:
DATASET = 'LINUX'

train_data = GEDDataset('/tmp/', name = "LINUX")
test_data = GEDDataset('/tmp/', name = "LINUX", train = False)
data = train_data + test_data

In [30]:
data[0]

Data(edge_index=[2, 18], i=[1], num_nodes=8)

In [31]:
node_labeled = False

In [32]:
generate_uniqe_graphs(data, DATASET, node_labeled)

Total number of graphs:  1000
Selecting Small Graphs...
Graphs with at most 20 nodes: 1000
Filtering isomorphic graphs...


100%|██████████| 999/999 [00:00<00:00, 1937.14it/s]


Total number of Unique graphs: 89
Maximum number of nodes in the dataset:  10
Splitting and dumping graphs...
Train: 53, Val: 17, Test: 19
sample: Data(edge_index=[2, 16], x=[9, 1])
saved LINUX into ./data_wo_node_attr/LINUX/train.pt
saved LINUX into ./data_wo_node_attr/LINUX/val.pt
saved LINUX into ./data_wo_node_attr/LINUX/test.pt


In [33]:
check_leakage(dest_folder +'/LINUX/', full_check = False)

Checking for leakage...
loading saved splits from  ./data_wo_node_attr//LINUX/
Number of graphs in train:  53  Val:  17  Test:  19


100%|██████████| 17/17 [00:00<00:00, 1142.50it/s]


Number of leaks in Val from train:  0


100%|██████████| 19/19 [00:00<00:00, 1171.92it/s]

Number of leaks in test from train:  0


In [34]:
dummy = torch.load(dest_folder +'/LINUX/train.pt')
dummy[0]

Data(edge_index=[2, 16], x=[9, 1])

# OGBG

In [35]:
from ogb.graphproppred import PygGraphPropPredDataset
from torch_geometric.data import DataLoader


## ogbg-code2

In [36]:
DATASET = 'ogbg-code2'
data = PygGraphPropPredDataset(name = DATASET) 

In [37]:
data[0]

Data(edge_index=[2, 243], x=[244, 2], node_is_attributed=[244, 1], node_dfs_order=[244, 1], node_depth=[244, 1], y=[1], num_nodes=244)

In [38]:
# For OGBA-CODE2 Label adjustment

node_labeled = False

valid_graphs = []
for g in tqdm(data):
    if (g.num_nodes <= 20):
        labels = g.x[:, 0]
        g.x = torch.nn.functional.one_hot(labels, 97).float()
        rev1 = torch.stack((g.edge_index[1], g.edge_index[0]), 0)
        g.edge_index = torch.cat((g.edge_index, rev1), -1)
        valid_graphs.append(g)
data = valid_graphs
print("Total number valid graphs in the dataset: ", len(data))

100%|██████████| 452741/452741 [00:53<00:00, 8459.43it/s]


Total number valid graphs in the dataset:  404


In [39]:
%%time
generate_uniqe_graphs(data, DATASET, node_labeled)

Total number of graphs:  404
Selecting Small Graphs...
Graphs with at most 20 nodes: 404
Filtering isomorphic graphs...


 77%|███████▋  | 310/403 [00:15<00:06, 13.85it/s] 

Timeout for graph 269


100%|██████████| 403/403 [00:30<00:00, 13.05it/s]


Timeout for graph 369
Total number of Unique graphs: 128
Maximum number of nodes in the dataset:  20
Splitting and dumping graphs...
Train: 76, Val: 25, Test: 27
sample: Data(edge_index=[2, 38], x=[20, 1])
saved ogbg-code2 into ./data_wo_node_attr/ogbg-code2/train.pt
saved ogbg-code2 into ./data_wo_node_attr/ogbg-code2/val.pt
saved ogbg-code2 into ./data_wo_node_attr/ogbg-code2/test.pt
CPU times: user 31.5 s, sys: 1.19 s, total: 32.7 s
Wall time: 31.1 s


In [40]:
check_leakage(dest_folder + '/ogbg-code2', full_check = False)

Checking for leakage...
loading saved splits from  ./data_wo_node_attr//ogbg-code2
Number of graphs in train:  76  Val:  25  Test:  27


100%|██████████| 25/25 [00:00<00:00, 538.45it/s]


Number of leaks in Val from train:  0


100%|██████████| 27/27 [00:00<00:00, 664.58it/s]

Number of leaks in test from train:  0


In [41]:
dummy = torch.load(dest_folder + '/ogbg-code2/train.pt')
dummy[0]

Data(edge_index=[2, 38], x=[20, 1])

## ogbg-molhiv

In [42]:
DATASET = 'ogbg-molhiv'
data = PygGraphPropPredDataset(name = DATASET) 

In [43]:
print(len(data))
print(data[0])

41127
Data(edge_index=[2, 40], edge_attr=[40, 3], x=[19, 9], y=[1, 1], num_nodes=19)


In [44]:
node_labeled = False

In [45]:
new_graphs = []
cnt = 0
for g in data:
    if (g.num_nodes <= 50):
        labels = g.x[:, 0]
        g.x = torch.nn.functional.one_hot(labels, 119).float()
        new_graphs.append(g)
data = new_graphs


In [46]:
len(data)

39650

In [47]:
generate_uniqe_graphs(data, DATASET, node_labeled, max_unique_cnt = 1000)

Total number of graphs:  39650
Selecting Small Graphs...
Graphs with at most 20 nodes: 14923
Filtering isomorphic graphs...


  8%|▊         | 1204/14922 [00:06<01:13, 185.92it/s]


Total number of Unique graphs: 1000
Maximum number of nodes in the dataset:  20
Splitting and dumping graphs...
Train: 600, Val: 200, Test: 200
sample: Data(edge_index=[2, 34], x=[16, 1])
saved ogbg-molhiv into ./data_wo_node_attr/ogbg-molhiv/train.pt
saved ogbg-molhiv into ./data_wo_node_attr/ogbg-molhiv/val.pt
saved ogbg-molhiv into ./data_wo_node_attr/ogbg-molhiv/test.pt


In [48]:
dummy = torch.load(dest_folder + '/ogbg-molhiv/train.pt')
dummy[0]

Data(edge_index=[2, 34], x=[16, 1])

In [49]:
check_leakage(dest_folder + '/ogbg-molhiv', full_check = False)

Checking for leakage...
loading saved splits from  ./data_wo_node_attr//ogbg-molhiv
Number of graphs in train:  600  Val:  200  Test:  200


100%|██████████| 200/200 [00:01<00:00, 158.47it/s]


Number of leaks in Val from train:  0


100%|██████████| 200/200 [00:01<00:00, 162.99it/s]


Number of leaks in test from train:  0


## ogbg-molpcba

In [50]:
DATASET = 'ogbg-molpcba'
data = PygGraphPropPredDataset(name = DATASET) 

In [51]:
print(len(data))
print(data[0])

437929
Data(edge_index=[2, 44], edge_attr=[44, 3], x=[20, 9], y=[1, 128], num_nodes=20)


In [52]:
node_labeled = False

In [53]:
new_graphs = []
cnt = 0
for g in data:
    if (g.num_nodes <= 20):
        labels = g.x[:, 0]
        g.x = torch.nn.functional.one_hot(labels, 119).float()
        new_graphs.append(g)
data = new_graphs
len(data)

80355

In [54]:
generate_uniqe_graphs(data, DATASET, node_labeled, max_unique_cnt = 1000)

Total number of graphs:  80355
Selecting Small Graphs...
Graphs with at most 20 nodes: 80355
Filtering isomorphic graphs...


  1%|▏         | 1035/80354 [00:08<10:46, 122.75it/s]


Total number of Unique graphs: 1000
Maximum number of nodes in the dataset:  20
Splitting and dumping graphs...
Train: 600, Val: 200, Test: 200
sample: Data(edge_index=[2, 36], x=[17, 1])
saved ogbg-molpcba into ./data_wo_node_attr/ogbg-molpcba/train.pt
saved ogbg-molpcba into ./data_wo_node_attr/ogbg-molpcba/val.pt
saved ogbg-molpcba into ./data_wo_node_attr/ogbg-molpcba/test.pt


In [55]:
dummy = torch.load(dest_folder + '/ogbg-molpcba/train.pt')
dummy[0]

Data(edge_index=[2, 36], x=[17, 1])

In [56]:
check_leakage(dest_folder + '/ogbg-molpcba', full_check = False)

Checking for leakage...
loading saved splits from  ./data_wo_node_attr//ogbg-molpcba
Number of graphs in train:  600  Val:  200  Test:  200


100%|██████████| 200/200 [00:01<00:00, 104.88it/s]


Number of leaks in Val from train:  0


100%|██████████| 200/200 [00:01<00:00, 109.61it/s]

Number of leaks in test from train:  0


# Mutagenicity

In [64]:
DATASET = 'Mutagenicity'
data = TUDataset(root = '/tmp/', name = DATASET) 

In [65]:
print(len(data))
print(data[0])

4337
Data(edge_index=[2, 32], x=[16, 14], edge_attr=[32, 3], y=[1])


In [66]:
count = torch.sum(data[6].x, dim=-1)
print(count)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])


In [67]:
node_labeled = False

In [68]:
generate_uniqe_graphs(data, DATASET, node_labeled)

Total number of graphs:  4337
Selecting Small Graphs...
Graphs with at most 20 nodes: 1287
Filtering isomorphic graphs...


100%|██████████| 1286/1286 [00:09<00:00, 140.75it/s]


Total number of Unique graphs: 729
Maximum number of nodes in the dataset:  20
Splitting and dumping graphs...
Train: 437, Val: 145, Test: 147
sample: Data(edge_index=[2, 16], x=[9, 1])
saved Mutagenicity into ./data_wo_node_attr/Mutagenicity/train.pt
saved Mutagenicity into ./data_wo_node_attr/Mutagenicity/val.pt
saved Mutagenicity into ./data_wo_node_attr/Mutagenicity/test.pt


In [69]:
dummy = torch.load(dest_folder + '/Mutagenicity/train.pt')
dummy[0]

Data(edge_index=[2, 16], x=[9, 1])

In [70]:
check_leakage(dest_folder + '/Mutagenicity', full_check = False)

Checking for leakage...
loading saved splits from  ./data_wo_node_attr//Mutagenicity
Number of graphs in train:  437  Val:  145  Test:  147


100%|██████████| 145/145 [00:01<00:00, 111.53it/s]


Number of leaks in Val from train:  0


100%|██████████| 147/147 [00:01<00:00, 103.24it/s]


Number of leaks in test from train:  0


# Yeast

In [77]:
DATASET = 'Yeast'
data = TUDataset(root = '/tmp/', name = DATASET) 

In [78]:
print(len(data))
print(data[0])

79601
Data(edge_index=[2, 32], x=[18, 74], edge_attr=[32, 3], y=[1])


In [79]:
count = torch.sum(data[1].x, dim=-1)
print(count)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])


In [80]:
node_labeled = False

In [81]:
generate_uniqe_graphs(data, DATASET, node_labeled, max_unique_cnt=1000)

Total number of graphs:  79601
Selecting Small Graphs...
Graphs with at most 20 nodes: 41724
Filtering isomorphic graphs...


  3%|▎         | 1115/41723 [00:08<05:19, 127.17it/s]


Total number of Unique graphs: 1000
Maximum number of nodes in the dataset:  20
Splitting and dumping graphs...
Train: 600, Val: 200, Test: 200
sample: Data(edge_index=[2, 36], x=[17, 1])
saved Yeast into ./data_wo_node_attr/Yeast/train.pt
saved Yeast into ./data_wo_node_attr/Yeast/val.pt
saved Yeast into ./data_wo_node_attr/Yeast/test.pt


In [82]:
dummy2 = torch.load(dest_folder + '/Yeast/train.pt')
print(dummy2[2])

Data(edge_index=[2, 12], x=[6, 1])


In [83]:
check_leakage(dest_folder + '/Yeast', full_check = False)

Checking for leakage...
loading saved splits from  ./data_wo_node_attr//Yeast
Number of graphs in train:  600  Val:  200  Test:  200


100%|██████████| 200/200 [00:01<00:00, 112.07it/s]


Number of leaks in Val from train:  0


100%|██████████| 200/200 [00:01<00:00, 114.00it/s]

Number of leaks in test from train:  0
